In [1]:
using RigidBodyDynamics
using StaticArrays
import Compat.Iterators: filter

INFO: Recompiling stale cache file /Users/twan/code/julia/RigidBodyDynamics/lib/v0.5/RigidBodyDynamics.ji for module RigidBodyDynamics.


In [2]:
# TODO: grab everything off of github and cache
DRAKE_DISTRO = "$(ENV["HOME"])/code/drake-distro"
urdf = "$DRAKE_DISTRO/drake/examples/Valkyrie/urdf/urdf/valkyrie_A_sim_drake_one_neck_dof_wide_ankle_rom.urdf"
mechanism = RigidBodyDynamics.parse_urdf(Float64, urdf);

In [3]:
# Add sole frame
ankleRoll = first(filter(j -> j.name == "leftAnkleRoll", joints(mechanism)))
soleFrame = CartesianFrame3D("l_foot_sole")
soleToFoot = Transform3D(soleFrame, ankleRoll.frameAfter, SVector(0.067, 0., -0.09))
add_body_fixed_frame!(mechanism, soleToFoot);

In [4]:
# Reroot at the left foot and remove fixed joints
pelvis = first(filter(b -> name(b) == "pelvis", bodies(mechanism)))
pelvisToWorld = first(filter(j -> j.name == "pelvis_to_world", joints(mechanism)))
leftFoot = first(filter(b -> name(b) == "leftFoot", bodies(mechanism)))
leftFootToWorld = Joint("leftFootToWorld", Fixed{Float64}())
world = root_body(mechanism)
attach!(mechanism, world, leftFootToWorld, Transform3D(leftFootToWorld.frameBefore, default_frame(world), -soleToFoot.trans), leftFoot)
remove_joint!(mechanism, pelvisToWorld)
remove_fixed_tree_joints!(mechanism);

In [5]:
mechanism

Spanning tree:
Vertex: world (root)
  Vertex: leftAnklePitchLink, Edge: leftAnkleRoll
    Vertex: leftKneePitchLink, Edge: leftAnklePitch
      Vertex: leftHipPitchLink, Edge: leftKneePitch
        Vertex: leftHipRollLink, Edge: leftHipPitch
          Vertex: leftHipYawLink, Edge: leftHipRoll
            Vertex: pelvis, Edge: leftHipYaw
              Vertex: torsoYawLink, Edge: torsoYaw
                Vertex: torsoPitchLink, Edge: torsoPitch
                  Vertex: torso, Edge: torsoRoll
                    Vertex: lowerNeckPitchLink, Edge: lowerNeckPitch
                    Vertex: rightShoulderPitchLink, Edge: rightShoulderPitch
                      Vertex: rightShoulderRollLink, Edge: rightShoulderRoll
                        Vertex: rightShoulderYawLink, Edge: rightShoulderYaw
                          Vertex: rightElbowPitchLink, Edge: rightElbowPitch
                            Vertex: rightForearmLink, Edge: rightForearmYaw
                              Vertex: rightWristRol